In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Parameters
NUM_FLIGHTS = 100
NUM_GATES = 25
AIRLINES = ['MU', 'CA', 'CZ', 'HU', 'ZH', 'HO']
TERMINALS = ['T1', 'T2']
AIRCRAFT_TYPES = ['A320', 'A330', 'B737', 'B777']
DELAY_PROB = 0.5
BUFFER_MIN = 20  # buffer time before and after usage

np.random.seed(42)

# Generate gate info
gate_list = []
for i in range(NUM_GATES):
    gate_id = f"G{i+1:02d}"
    is_bridge = np.random.choice([1, 0], p=[0.7, 0.3])
    gate = {
        "Gate_ID": gate_id,
        "Terminal": random.choice(TERMINALS),
        "Is_Bridge": is_bridge,
        "Max_Aircraft": random.choice(AIRCRAFT_TYPES)
    }
    gate_list.append(gate)
gates_df = pd.DataFrame(gate_list)

# Prepare remote stands for non-bridge gates
remote_stands = [f"R{20+i:02d}" for i in range(NUM_GATES)]  # R20–R44
bridge_gates = gates_df[gates_df["Is_Bridge"] == 1]["Gate_ID"].tolist()
remote_gate_map = dict(zip(gates_df[gates_df["Is_Bridge"] == 0]["Gate_ID"], remote_stands))

# Generate flights
flight_list = []
base_time = datetime(2025, 8, 21, 6, 0, 0)
for _ in range(NUM_FLIGHTS):
    arr_time = base_time + timedelta(minutes=np.random.randint(0, 720))
    dep_time = arr_time + timedelta(minutes=np.random.randint(40, 90))
    airline = random.choice(AIRLINES)
    flight = {
        "Flight_ID": f"{airline}{np.random.randint(1000, 9999)}",
        "Aircraft_Type": random.choice(AIRCRAFT_TYPES),
        "Scheduled_ARR": arr_time,
        "Scheduled_DEP": dep_time,
        "Airline": airline,
        "Terminal": random.choice(TERMINALS),
        "Is_International": np.random.choice([0, 1], p=[0.8, 0.2]),
        "Delay_MIN": np.random.randint(10, 120) if np.random.rand() < DELAY_PROB else 0,
    }
    flight_list.append(flight)
flights_df = pd.DataFrame(flight_list)

# Compute actual arrival time
flights_df["Actual_ARR"] = flights_df["Scheduled_ARR"] + pd.to_timedelta(flights_df["Delay_MIN"], unit='m')

# Sort flights by arrival time for sequential assignment
flights_df = flights_df.sort_values("Actual_ARR").reset_index(drop=True)

# Track stand usage: {stand_id: [(start_time, end_time), ...]}
stand_usage = {}

# Assign gates and stands
assigned_gates = []
assigned_stands = []

for idx, row in flights_df.iterrows():
    # Use scheduled time for advance planning
    arr = row["Scheduled_ARR"] - timedelta(minutes=BUFFER_MIN)
    dep = row["Scheduled_DEP"] + timedelta(minutes=BUFFER_MIN)

    assigned = False
    for _, gate_row in gates_df.iterrows():
        gate_id = gate_row["Gate_ID"]
        is_bridge = gate_row["Is_Bridge"]

        # Determine stand ID
        stand_id = gate_id if is_bridge else remote_gate_map[gate_id]

        # Check for time conflicts
        usage = stand_usage.get(stand_id, [])
        conflict = any((arr < end and dep > start) for start, end in usage)
        if not conflict:
            assigned_gates.append(gate_id)
            assigned_stands.append(stand_id)
            stand_usage.setdefault(stand_id, []).append((arr, dep))
            assigned = True
            break

    if not assigned:
        assigned_gates.append(None)
        assigned_stands.append(None)
        print(f"❌ Flight {row['Flight_ID']} cannot be assigned.")

# Add assignments to DataFrame
flights_df["Assigned_Gate"] = assigned_gates
flights_df["Assigned_Stand"] = assigned_stands

# Final output
output_cols = [
    "Flight_ID", "Aircraft_Type", "Airline", "Is_International",
    "Scheduled_ARR", "Scheduled_DEP", "Delay_MIN", "Actual_ARR",
    "Assigned_Gate", "Assigned_Stand"
]
print(flights_df[output_cols].head(10))


  Flight_ID Aircraft_Type Airline  Is_International       Scheduled_ARR  \
0    CA4157          A320      CA                 1 2025-08-21 06:35:00   
1    HU4420          A320      HU                 0 2025-08-21 06:36:00   
2    MU5380          B737      MU                 0 2025-08-21 06:42:00   
3    MU3491          B737      MU                 0 2025-08-21 06:53:00   
4    HU7316          B777      HU                 0 2025-08-21 06:57:00   
5    MU1728          A320      MU                 0 2025-08-21 07:01:00   
6    MU5491          A320      MU                 0 2025-08-21 07:08:00   
7    MU9392          B777      MU                 0 2025-08-21 06:27:00   
8    HO2571          A320      HO                 0 2025-08-21 07:38:00   
9    MU8390          A320      MU                 0 2025-08-21 07:39:00   

        Scheduled_DEP  Delay_MIN          Actual_ARR Assigned_Gate  \
0 2025-08-21 07:27:00          0 2025-08-21 06:35:00           G01   
1 2025-08-21 07:39:00          0 2

In [ ]:
# Select and rename relevant columns for export
export_df = flights_df[[
    "Flight_ID", "Aircraft_Type", "Airline", "Is_International",
    "Scheduled_ARR", "Scheduled_DEP", "Delay_MIN", "Actual_ARR",
    "Assigned_Gate", "Assigned_Stand"
]].copy()

# Sort by Scheduled_ARR for easier viewing
export_df.sort_values("Scheduled_ARR", inplace=True)

# Save to CSV
export_df.to_csv("simulated_flight_gate_assignment.csv", index=False, encoding="utf-8-sig")

print("Exported to simulated_flight_gate_assignment.csv")


Exported to simulated_flight_gate_assignment.csv


In [ ]:
from collections import Counter

usage_counter = Counter(assigned_stands)
for stand, count in sorted(usage_counter.items()):
    print(f"{stand}: used {count} times")

unassigned = sum(pd.isna(flights_df["Assigned_Stand"]))
print(f"\nUnassigned flights: {unassigned}")


G01: used 8 times
G04: used 6 times
G05: used 5 times
G06: used 6 times
G07: used 5 times
G09: used 4 times
G11: used 5 times
G14: used 4 times
G15: used 3 times
G16: used 4 times
G17: used 3 times
G18: used 3 times
G19: used 3 times
G20: used 2 times
G21: used 2 times
G22: used 3 times
G23: used 2 times
G24: used 1 times
G25: used 1 times
R20: used 6 times
R21: used 5 times
R22: used 6 times
R23: used 4 times
R24: used 5 times
R25: used 4 times

Unassigned flights: 0


In [ ]:
!pip install gymnasium


In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd
from datetime import timedelta


class RealTimeGateEnv(gym.Env):
    """
    Environment for real-time gate reallocation based on delays.
    Each step processes a flight; the agent decides to reassign it or not.
    """
    def __init__(self, flights_df, gates_df, buffer_min=20):
        super().__init__()
        self.flights_df = flights_df.copy()
        self.gates_df = gates_df.copy()
        self.buffer_min = buffer_min
        self.num_gates = len(gates_df)
        self.num_flights = len(flights_df)

        self.action_space = spaces.Discrete(self.num_gates + 1)  # 0~n-1 = reassign to gate; n = keep original
        self.observation_space = spaces.Dict({
            "flight_features": spaces.Box(low=0, high=1, shape=(6,), dtype=np.float32),
            "gate_mask": spaces.MultiBinary(self.num_gates)
        })

        self.remote_gate_map = self._build_remote_gate_map()
        self.current_idx = 0
        self.stand_usage = {g: [] for g in gates_df["Gate_ID"]}

    def _build_remote_gate_map(self):
        # Map non-bridge gates to remote stand IDs
        remote_stands = [f"R{20+i:02d}" for i in range(self.num_gates)]
        return dict(zip(self.gates_df[self.gates_df["Is_Bridge"] == 0]["Gate_ID"], remote_stands))

    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)
        self.current_idx = 0
        self.stand_usage = {g: [] for g in self.gates_df["Gate_ID"]}
        return self._get_obs(), {}

    def _get_obs(self):
        if self.current_idx >= self.num_flights:
            return {
                "flight_features": np.zeros(6, dtype=np.float32),
                "gate_mask": np.zeros(self.num_gates, dtype=np.int8)
            }

        row = self.flights_df.iloc[self.current_idx]
        arr = row["Actual_ARR"] - timedelta(minutes=self.buffer_min)
        dep = row["Scheduled_DEP"] + timedelta(minutes=self.buffer_min)

        gate_mask = []
        for _, gate_row in self.gates_df.iterrows():
            gate_id = gate_row["Gate_ID"]
            usage = self.stand_usage[gate_id]
            conflict = any((arr < end and dep > start) for start, end in usage)
            gate_mask.append(0 if conflict else 1)

        # Features: arrival, dep, delay, intl, wide-body, original_gate_id
        arr_min = row["Actual_ARR"].hour * 60 + row["Actual_ARR"].minute
        dep_min = row["Scheduled_DEP"].hour * 60 + row["Scheduled_DEP"].minute
        features = np.array([
            arr_min / 1440,
            dep_min / 1440,
            row["Delay_MIN"] / 180,
            row["Is_International"],
            int(row["Aircraft_Type"] in ["A330", "B777"]),
            0.0 if pd.isna(row["Assigned_Gate"]) else int(row["Assigned_Gate"][1:]) / self.num_gates
        ], dtype=np.float32)

        return {
            "flight_features": features,
            "gate_mask": np.array(gate_mask, dtype=np.int8)
        }

    def step(self, action):
        info = {}
        if self.current_idx >= self.num_flights:
            return self._get_obs(), 0.0, True, True, info

        row = self.flights_df.iloc[self.current_idx]
        arr = row["Actual_ARR"] - timedelta(minutes=self.buffer_min)
        dep = row["Scheduled_DEP"] + timedelta(minutes=self.buffer_min)

        assigned_gate = row["Assigned_Gate"]
        gate_id = None
        reward = 0

        if action == self.num_gates:
            # Keep original gate
            gate_id = assigned_gate
        else:
            gate_id = self.gates_df.iloc[action]["Gate_ID"]

        # Check conflict
        usage = self.stand_usage.get(gate_id, [])
        conflict = any((arr < end and dep > start) for start, end in usage)

        if conflict or pd.isna(gate_id):
            reward = -10
        else:
            reward = 1 if gate_id == assigned_gate else 0.5  # encourage reuse
            self.stand_usage[gate_id].append((arr, dep))

        self.current_idx += 1
        terminated = self.current_idx >= self.num_flights
        truncated = False
        return self._get_obs(), reward, terminated, truncated, info

    def render(self):
        print(f"Flight {self.current_idx}")


In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

# Create and wrap environment
env = RealTimeGateEnv(flights_df, gates_df)
vec_env = DummyVecEnv([lambda: env])

# Train agent
model = PPO("MultiInputPolicy", vec_env, verbose=1)
model.learn(total_timesteps=20000)

# Test agent
# Test agent & collect reassignment results
obs, _ = env.reset()
total_reward = 0
reassigned_flights = []

while True:
    action, _ = model.predict(obs)
    obs, reward, terminated, truncated, _ = env.step(action)
    total_reward += reward

    # Collect reallocation info
    if env.current_idx <= env.num_flights:
        flight_row = env.flights_df.iloc[env.current_idx - 1]  # current flight
        assigned_gate = flight_row["Assigned_Gate"] if action == env.num_gates else env.gates_df.iloc[action]["Gate_ID"]
        stand_id = assigned_gate if assigned_gate in env.stand_usage else env.remote_gate_map.get(assigned_gate, None)

        reassigned_flights.append({
            "Flight_ID": flight_row["Flight_ID"],
            "Assigned_Gate_Reassigned": assigned_gate,
            "Assigned_Stand_Reassigned": stand_id
        })

    if terminated or truncated:
        break

print(f"✅ Total reward from real-time reallocation: {total_reward}")

# Convert collected info to reassigned_df
reassigned_df = pd.DataFrame(reassigned_flights)


Using cpu device
-----------------------------
| time/              |      |
|    fps             | 297  |
|    iterations      | 1    |
|    time_elapsed    | 6    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 263         |
|    iterations           | 2           |
|    time_elapsed         | 15          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007993128 |
|    clip_fraction        | 0.0342      |
|    clip_range           | 0.2         |
|    entropy_loss         | -3.25       |
|    explained_variance   | 0.00267     |
|    learning_rate        | 0.0003      |
|    loss                 | 363         |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0145     |
|    value_loss           | 1.14e+03    |
-----------------------------------------
-----------------

In [ ]:
import pandas as pd

# 1. Filter only delayed flights
delayed = flights_df[flights_df['Delay_MIN'] > 0].copy()

# 2. Merge reassigned gate/stand into delayed flights
delayed = delayed.merge(
    reassigned_df,
    on='Flight_ID',
    how='left'
)

# 3. Check whether reassignment occurred
def was_reassigned(row):
    return (
        row['Assigned_Gate'] != row['Assigned_Gate_Reassigned'] or
        row['Assigned_Stand'] != row['Assigned_Stand_Reassigned']
    )

delayed['Reassigned'] = delayed.apply(was_reassigned, axis=1)

# 4. Stats
total_delayed = len(delayed)
total_reassigned = delayed['Reassigned'].sum()
reassignment_rate = total_reassigned / total_delayed * 100 if total_delayed else 0

print(f"Total Delayed Flights: {total_delayed}")
print(f"Reassigned Flights: {total_reassigned}")
print(f"Reassignment Rate: {reassignment_rate:.2f}%")

# 5. Export (optional)
delayed.to_csv("delayed_flight_reassignment_analysis.csv", index=False)
print("✅ CSV file saved as 'delayed_flight_reassignment_analysis.csv'")



Total Delayed Flights: 47
Reassigned Flights: 47
Reassignment Rate: 100.00%
✅ CSV file saved as 'delayed_flight_reassignment_analysis.csv'
